## SageMaker Model and Model Registry 
- haimtran 01/06/2023
- [Model Artifact](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-marketplace-develop.html)
- [SageMaker SDK Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html)
- [Retrieve Pre-Built SageMaker Docker Image](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html)
- [SageMaker Docker Registry Paths](https://docs.aws.amazon.com/sagemaker/latest/dg-ecr-paths/ecr-ap-southeast-1.html#pca-ap-southeast-1.title)

A deployable model consits of at least 
- Image URI 
- ModelDataURL after training 
- Role 

In [ ]:
# !python -m pip install sagemaker
# !python -m pip install boto3

In [ ]:
import sagemaker
import boto3

In [ ]:
session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = session.default_bucket()

In [ ]:
session._region_name

## Boto3 Check and Existing Model 

In [ ]:
sagemaker_client = boto3.client("sagemaker")

In [ ]:
sagemaker_client.list_models()

In [ ]:
sagemaker_client.describe_model(
    ModelName="PCAModel1aac2da2-df8c-4da8-8097-5cb336dae838"
)

## SageMaker SDK: Create a Model 

In [ ]:
from sagemaker.model import Model
from sagemaker.image_uris import retrieve

In [ ]:
image_uri = retrieve(
    framework="pca",
    region=session._region_name, 
    instance_type="ml.m4.xlarge",
    version="0.23-1"
)

In [ ]:
print(image_uri)

In [ ]:
model = Model(
    image_uri=image_uri,
    model_data="s3://sagemaker-ap-southeast-1-394599967678/TrainingJobName27a61f71-876d-4d88-a39f-487cd1d1ebd4/output/model.tar.gz",
    role=role,
    name="PCACreatedFromNoteBookModel"
)

In [ ]:
model.create(instance_type="ml.m4.xlarge")

In [ ]:
# model.delete_model()

## Deploy a Model as a Endpoint 

[deploy a model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html), and it would take about 5 minutes bofore the endpoint ready 

In [ ]:
model.deploy(
    initial_instance_count=1, 
    instance_type="ml.m4.xlarge"
)

describe the deployed endpoint 

In [ ]:
sagemaker_client.list_endpoints()

In [ ]:
sagemaker_client.describe_endpoint(
    EndpointName="PCACreatedFromNoteBookModel-2023-06-01-05-26-19-579"
)